## Docelowa struktura rekordów:
1. Tabela: Obwody
    * id
    * numer
    * lat
    * lon
2. Tabela: Wyniki_Prezydent
    * rok
    * obwód
    * Majchrowski
    * kandydat PO
    * kandydat PiS
    * Gibała
    * Leśniak
    * Berkowicz
    * Inni
    * wyborcy
    * oddane głosy
3. Tabela: Wyniki_rada
    * rok
    * obwód
    * Majchrowski
    * PO
    * PiS
    * Gibała
    * Lewica
    * Berkowicz
    * Inni
    * wyborcy
    * oddane głosy



Założenie 1:
W analizie zostana pominiete komisje zamknięte ze względu na małą liczbę głosów (<1%) oraz brak znaczenia dla terytorialnego rozkładu głosów

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from lxml import html
from lxml.cssselect import CSSSelector
import requests
import pandas as pd
import matplotlib.pyplot as plt
import geocoder
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.colors as colors
import scipy.stats 
% matplotlib inline

### Web scraping i data cleaning - obwody wyborcze

In [4]:
obwody2006=pd.read_csv("obwody2006.csv")

In [5]:
obwody2006.head()

,Nr,Siedziba,Typ,Unnamed: 3,Granice obwodu głosowania
0,1,"Gimnazjum Nr 3, Kraków Wąska 5",Powszechny,Nie,"Dajwór parzyste od 2 do 20, Halicka (bez 21 i ..."
1,2,"Gimnazjum Nr 3, Kraków Wąska 5",Powszechny,Nie,"Bocheńska, Bonifraterska, Gazowa, Krakowska pa..."
2,3,Staromiejskie Centrum Kultury Młodzieży Klub Ś...,Powszechny,Nie,Bożego Ciała nieparzyste od 5 do 31 i parzyste...
3,4,"Szkoła Podstawowa Nr 11, Kraków Miodowa 36",Powszechny,Nie,"Bartosza, Brzozowa parzyste od 12 do 22, Ciemn..."
4,5,"Szkoła Podstawowa Nr 11, Kraków Miodowa 36",Powszechny,Nie,"Dietla nieparzyste od 77 do 115, Joselewicza p..."


In [6]:
obwody2010=pd.read_csv("obwody2010.csv")

In [7]:
obwody2010.head()

,Unnamed: 0,nr,adres,typ,niep,granice
0,0,1,"Gimnazjum Nr 3, ul. Wąska 5, Kraków",Powszechny,Nie,"Dajwór parzyste od 2 do 20, Halicka (bez 21 i ..."
1,1,2,"Gimnazjum Nr 3, ul.Wąska 5, Kraków",Powszechny,Nie,"Bocheńska, Bonifraterska, Gazowa, Krakowska pa..."
2,2,3,Staromiejskie Centrum Kultury Młodzieży Klub Ś...,Powszechny,Nie,Bożego Ciała nieparzyste od 5 do 31 i parzyste...
3,3,4,"Szkoła Podstawowa Nr 11, ul. Miodowa 36, Kraków",Powszechny,Nie,"Bartosza, Brzozowa parzyste od 12 do 22, Ciemn..."
4,4,5,"Szkoła Podstawowa Nr 11, ul. Miodowa 36, Kraków",Powszechny,Nie,"Dietla nieparzyste od 77 do 115, Joselewicza p..."


In [8]:
obwody2014=pd.read_csv("obwody2014.csv")

In [9]:
obwody2014.head()

,numer,Adres,Granice obwodu
0,1,"Kraków-Śródmieście, Stare Miasto ul. Wąska 5","DAJWÓR parzyste od 2 do 20, HALICKA (bez 21, 2..."
1,2,"Kraków-Śródmieście, Stare Miasto ul.Wąska 5","BOCHEŃSKA, BONIFRATERSKA, GAZOWA, KRAKOWSKA pa..."
2,3,"Kraków-Śródmieście, Stare Miasto ul. J. Dietla 53",BOŻEGO CIAŁA nieparzyste od 5 do 31 i parzyste...
3,4,"Kraków-Śródmieście, Stare Miasto ul. Miodowa 36","BARTOSZA, BRZOZOWA parzyste od 12 do 22, CIEMN..."
4,5,"Kraków-Śródmieście, Stare Miasto ul. Miodowa 36","DIETLA nieparzyste od 77 do 115, JOSELEWICZA p..."


In [10]:
obwody2006['granice']=obwody2006["Granice obwodu głosowania"].str.lower()

In [11]:
obwody2006=obwody2006[obwody2006["Typ"]=="Powszechny"]

In [12]:
obwody2006['nr']=obwody2006["Nr"]

In [13]:
obwody2006=obwody2006.drop(["Nr","Siedziba","Typ","Unnamed: 3", "Granice obwodu głosowania"], axis=1)

In [14]:
obwody2006.head()

,granice,nr
0,"dajwór parzyste od 2 do 20, halicka (bez 21 i ...",1
1,"bocheńska, bonifraterska, gazowa, krakowska pa...",2
2,bożego ciała nieparzyste od 5 do 31 i parzyste...,3
3,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4
4,"dietla nieparzyste od 77 do 115, joselewicza p...",5


In [15]:
obwody2010['granice']=obwody2010['granice'].str.lower()

In [16]:
obwody2010=obwody2010[obwody2010['typ']=='Powszechny']

In [17]:
obwody2010=obwody2010.drop(["Unnamed: 0","adres","typ","niep"],axis=1)

In [18]:
obwody2010.head()

,nr,granice
0,1,"dajwór parzyste od 2 do 20, halicka (bez 21 i ..."
1,2,"bocheńska, bonifraterska, gazowa, krakowska pa..."
2,3,bożego ciała nieparzyste od 5 do 31 i parzyste...
3,4,"bartosza, brzozowa parzyste od 12 do 22, ciemn..."
4,5,"dietla nieparzyste od 77 do 115, joselewicza p..."


In [19]:
obwody2014['granice']=obwody2014['Granice obwodu'].str.lower()

In [20]:
obwody2014["nr"]=obwody2014["numer"]

In [21]:
obwody2014=obwody2014.drop(["numer","Adres","Granice obwodu"], axis=1)

In [22]:
obwody2014.head()

,granice,nr
0,"dajwór parzyste od 2 do 20, halicka (bez 21, 2...",1
1,"bocheńska, bonifraterska, gazowa, krakowska pa...",2
2,bożego ciała nieparzyste od 5 do 31 i parzyste...,3
3,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4
4,"dietla nieparzyste od 77 do 115, joselewicza p...",5


In [24]:
from difflib import SequenceMatcher
def mapping(source, target):
    obwody=[]
    for idx1, target_row in target.iterrows():
        mapping=[]
        for idx2, source_row in source.iterrows():        
            similarity=SequenceMatcher(None,source_row['granice'],target_row['granice']).ratio()
            if similarity>0.85:
                mapping=[[source_row['nr'],similarity]]
                break
            else:
                mapping.append([source_row['nr'],similarity])
        if len(mapping)>3:
            mapping=sorted(mapping, key=lambda row: row[1], reverse=True)
            if mapping[0][1]-mapping[1][1]>0.2:
                mapping=[mapping[0]]
            else:
                mapping=mapping[:3]
        obwody.append(mapping)
    return obwody
obwody2010['obwody2006']=mapping(obwody2006,obwody2010)
obwody2014['obwody2006']=mapping(obwody2006,obwody2014)  

In [25]:
obwody2010.head()

,nr,granice,obwody2006
0,1,"dajwór parzyste od 2 do 20, halicka (bez 21 i ...","[[1, 0.9957805907172996]]"
1,2,"bocheńska, bonifraterska, gazowa, krakowska pa...","[[2, 1.0]]"
2,3,bożego ciała nieparzyste od 5 do 31 i parzyste...,"[[3, 1.0]]"
3,4,"bartosza, brzozowa parzyste od 12 do 22, ciemn...","[[4, 1.0]]"
4,5,"dietla nieparzyste od 77 do 115, joselewicza p...","[[5, 1.0]]"


In [26]:
map_06_10=[[97,97],[98,97],[101,100],[162,160],[170,169],[172,170],[176,173],[177,173],[202,197],[213,207],[217,210],[226,219],
 [237,230],[310,301],[316,310],[317,309],[324,314],[330,320],[331,320],[392,381],[408,397]]

In [27]:
map_06_14=[[14,14],[17,17],[44,45],[57,57],[64,63],[65,63],[69,68],[81,78],[82,79],[90,88],[91,88],[93,90],[98,94],[99,95],[101,97],
 [102,97],[103,98],[106,100],[107,101],[118,111],[121,114],[122,115],[123,115],[124,115],[140,132],[158,149],[159,149],
 [162,152],[179,169],[180,169],[181,170],[182,170],[183,170],[184,171],[186,173],[187,173],[189,175],[194,179],[198,181],
 [205,188],[206,189],[207,189],[211,193],[212,194],[216,197],[217,198],[218,199],[228,207],[232,211],[234,169],[236,169],
 [238,213],[240,215],[241,216],[242,217],[244,219],[251,226],[252,227],[255,230],[257,231],[261,234],[267,240],[268,240],
 [270,241],[279,249],[286,257],[290,261],[293,264],[298,267],[305,276],[307,278],[308,278],[309,279],[310,280],[311,280],
 [329,298],[332,301],[333,301],[336,303],[338,305],[340,306],[344,309],[347,312],[348,313],[349,314],[355,320],[356,320],
 [372,336],[373,337],[376,340],[381,344],[385,348],[393,356],[397,360],[398,361],[400,362],[401,362],[405,366],[420,381],
 [424,385],[425,384],[436,397],[438,403]]


In [28]:
def clean_obwody06_10(row):  
    obwod=None
    if len(row['obwody2006'])>1:
        for item in map_06_10:
            if item[0]==row['nr']:               
                obwod=item[1]
                break
    else:
        obwod=row['obwody2006'][0][0]    
    return obwod
obwody2010['obwod06']=obwody2010.apply(clean_obwody06_10, axis=1)

In [29]:
obwody2010.head()

,nr,granice,obwody2006,obwod06
0,1,"dajwór parzyste od 2 do 20, halicka (bez 21 i ...","[[1, 0.9957805907172996]]",1
1,2,"bocheńska, bonifraterska, gazowa, krakowska pa...","[[2, 1.0]]",2
2,3,bożego ciała nieparzyste od 5 do 31 i parzyste...,"[[3, 1.0]]",3
3,4,"bartosza, brzozowa parzyste od 12 do 22, ciemn...","[[4, 1.0]]",4
4,5,"dietla nieparzyste od 77 do 115, joselewicza p...","[[5, 1.0]]",5


In [30]:
def clean_obwody06_14(row):  
    obwod=None
    if len(row['obwody2006'])>1:
        for item in map_06_14:
            if item[0]==row['nr']:               
                obwod=item[1]
                break
    else:
        obwod=row['obwody2006'][0][0]    
    return obwod
obwody2014['obwod06']=obwody2014.apply(clean_obwody06_14, axis=1)

In [31]:
obwody2014.head()

,granice,nr,obwody2006,obwod06
0,"dajwór parzyste od 2 do 20, halicka (bez 21, 2...",1,"[[1, 0.9723991507430998]]",1
1,"bocheńska, bonifraterska, gazowa, krakowska pa...",2,"[[2, 1.0]]",2
2,bożego ciała nieparzyste od 5 do 31 i parzyste...,3,"[[3, 1.0]]",3
3,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4,"[[4, 1.0]]",4
4,"dietla nieparzyste od 77 do 115, joselewicza p...",5,"[[5, 1.0]]",5


In [32]:
len(obwody2010[obwody2010['obwod06'].isnull()])

0

In [33]:
len(obwody2014[obwody2014['obwod06'].isnull()])

0

In [34]:
obwody2010=obwody2010.drop('obwody2006', axis=1)
obwody2014=obwody2014.drop('obwody2006', axis=1)

In [35]:
obwody2010.head()

,nr,granice,obwod06
0,1,"dajwór parzyste od 2 do 20, halicka (bez 21 i ...",1
1,2,"bocheńska, bonifraterska, gazowa, krakowska pa...",2
2,3,bożego ciała nieparzyste od 5 do 31 i parzyste...,3
3,4,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4
4,5,"dietla nieparzyste od 77 do 115, joselewicza p...",5


In [36]:
obwody2014.head()

,granice,nr,obwod06
0,"dajwór parzyste od 2 do 20, halicka (bez 21, 2...",1,1
1,"bocheńska, bonifraterska, gazowa, krakowska pa...",2,2
2,bożego ciała nieparzyste od 5 do 31 i parzyste...,3,3
3,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4,4
4,"dietla nieparzyste od 77 do 115, joselewicza p...",5,5


#### uzupelnienie współrzędnych obwodów

In [37]:
obwody2006.head()

,granice,nr
0,"dajwór parzyste od 2 do 20, halicka (bez 21 i ...",1
1,"bocheńska, bonifraterska, gazowa, krakowska pa...",2
2,bożego ciała nieparzyste od 5 do 31 i parzyste...,3
3,"bartosza, brzozowa parzyste od 12 do 22, ciemn...",4
4,"dietla nieparzyste od 77 do 115, joselewicza p...",5


In [58]:
latlons=[]

def get_latlon(granice_split, latlons, latlon):
    adres=granice_split[0]    
    adres=adres.replace(" parzyste","")
    adres=adres.replace(" nieparzyste","")
    adres=adres.replace("dom pomocy społecznej ","")
    adres =adres.replace("kraków","")   
    adres_split=adres.split(" ")
    clean_adres=""
    for idx, item in enumerate(adres_split):
        if item=="od":
            clean_adres+=" "+adres_split[idx+1]
            break
        elif (item=="(bez") or (item=="/bez") or (item=="i"):
            break
        else:
            clean_adres+=" "+item  
             
    try:
        if clean_adres==" ":
            pass
        else:
            g=geocoder.osm(clean_adres+", Kraków")             
            lat=str(g.json['lat'])
            lon=str(g.json['lng'])
            latlon=lat+";"+lon
        try:
            if latlon in latlons:
                if len(granice_split)>1:
                    granice_split=granice_split[1:]
                    latlon=get_latlon(granice_split,latlons,latlon)                    
                else:
                    #print(clean_adres, latlon)
                    if clean_adres==" kobierzyńska 16":
                        lat='50.0363220'
                        lon='19.9288740'
                        latlon=lat+";"+lon 
                    elif clean_adres==" kurczaba":
                        lat='50.0133460'
                        lon='20.0057700'
                        latlon=lat+";"+lon
                    elif clean_adres==" osiedle strusia 7":
                        lat='50.0886040'
                        lon='20.0111560'
                        latlon=lat+";"+lon  
                    else:
                        lat='100'
                        lon='100'
                        latlon=lat+";"+lon   
        except KeyError:
            pass
    except ValueError:
        if len(granice_split)>1:
            granice_split=granice_split[1:]
            latlon=get_latlon(granice_split, latlons, latlon)
        else:
            if clean_adres=="kobierzyńska 91":
                lat='50.0275050'
                lon='19.9136340'
                latlon=lat+";"+lon 
            else:
                lat='0'
                lon='0'
                latlon=lat+";"+lon        
    latlons.append(latlon)
    print(clean_adres, latlon)
    return latlon

def get_geocode(granice):    
    granice_split=granice.split(",")
    latlon="0"
    latlon=get_latlon(granice_split, latlons, latlon)       
    return latlon


In [59]:
#obwody2006['latlon']=obwody2006['granice'].apply(get_geocode)
#obwody2010['latlon']=obwody2010['granice'].apply(get_geocode)
obwody2014['latlon']=obwody2014['granice'].apply(get_geocode)

 dajwór 2 50.0523429;19.9491997
 bocheńska 50.0485635;19.9450735
 bożego ciała 5 50.0518009;19.9431755
 bartosza 50.0504607;19.9487017
 dietla 77 50.055094;19.9448138
 bożego ciała 1 50.0522478;19.9427287
 augustiańska 7 50.0492845;19.9409054985253
 augustiańska 1 50.050684;19.9408462
 bernardyńska 50.0539741;19.9389437
 dominikańska 50.0590535;19.9397261
 bogusławskiego 1 50.0565803;19.9423314
 bogusławskiego 2 50.056625;19.9425572
 bonerowska 50.059063;19.9457319
 doktora zamenhofa  50.0635487;19.9468574
 floriańska 50.063437;19.9402823
 bracka 50.0605924;19.9363314
 plac na groblach 50.0561004;19.9320048
 aleja krasińskiego 1 50.0590085;19.9257049
 bożego miłosierdzia 50.0586044;19.9298349
  cybulskiego 50.0620171;19.9272761
 aleja mickiewicza 1 50.0620171;19.9272761
 czapskich 50.0605941;19.9300795
  czarnowiejska 1 50.0653309;19.9274954
 aleja mickiewicza 25 50.0653309;19.9274954
  grabowskiego 50.0670168;19.9276013
 aleja mickiewicza 41 50.0670168;19.9276013
 aleja słowackiego 1 

 kobierzyńska 91 100;100
 grota roweckiego 35 50.0285947;19.9117857
 gronostajowa 50.0279466;19.9031162
 baczyńskiego 50.0035284;19.8694379
 batalionów chłopskich 50.0156693;19.8711916
 błotniska 49.9884486;19.8571866
 benedyktyńska 50.0172788;19.8066348
 bagienna 50.0082239;19.838952
 bobrowa 50.0271349;19.856715
 bażanki 50.0326383;19.8920872
 ekielskiego 50.0312669;19.8986494
 do groty 50.0399225;19.904332
 biała droga 50.0495985;19.9223537
 ceglarska 50.0366428;19.9254223
 bałuckiego 50.049994;19.9260913
 barska 1 50.0520137;19.9281626
 kobierzyńska 16 50.0363220;19.9288740
 kapelanka 39 50.0338859;19.9255767
 grota roweckiego 11 50.0313015;19.920202
 grota roweckiego 31 50.0296645;19.9139978
 barska 47 50.0461436;19.9323637
 komandosów 1 50.0456699;19.9307045
 komandosów 10 50.0432944;19.9305723
 słomiana 2 50.0440905;19.9252295
 kapelanka 3 50.04282175;19.924169395848
 dekarzy 50.0214708;19.9217907
 chłodna 50.0180811;19.9268699
 borkowskie błonia 50.0137472;19.9111732
 goryczkow

 osiedle kalinowe 12 50.0924453;20.0150135750106
 osiedle kalinowe 18 50.09040885;20.0116938716793
 osiedle niepodległości 1 50.0819273;20.0219261
 osiedle niepodległości 8 50.0804227;20.022558799573
 osiedle albertyńskie 12 50.08162075;20.0178184941501
 małego księcia 50.0817517;20.0180134
 dąbrowskiej 17 50.0815585;20.0156963
 osiedle przy arce 1 50.0818867;20.0251801564868
 osiedle przy arce 9 50.0835399;20.0278392
 osiedle przy arce 13 50.0826523;20.0307930310993
 batalionu "/parasol"/ 50.0815482;20.0259424
 fatimska 50.0901873;20.0290275
 andersena 50.0834351;20.0411859
 generała okulickiego 49 50.0940114;20.022992
 osiedle jagiellońskie 1 50.08821655;20.0213690032567
 anny jagiellonki 50.0896407;20.0232359
 osiedle kazimierzowskie 26 50.0861578;20.0218973206413
 osiedle kazimierzowskie 1 50.0832412;20.0238274
 elżbiety łokietkówny 50.0853948;20.0218522
 kleinera 50.0806989;20.0608829
 osiedle na wzgórzach 1 50.09257655;20.0598748287866
 osiedle na wzgórzach 11 50.0955318;20.06072

In [62]:
obwody2006.iloc[415]
obwody2010[obwody2010['latlon']=="0;0"]
#bwody2006[obwody2006['latlon']=="50.0432373;19.9448962"]
#obwody2014['latlon'].value_counts()#[obwody2006['latlon'].value_counts()>1]



,nr,granice,obwod06,latlon


In [64]:
def split_lat(row):
    latlon=row.split(";")
    lat=latlon[0]
    return lat

def split_lon(row):
    latlon=row.split(";")
    lon=latlon[1]
    return lon
    

In [ ]:
obwody2006['lat']=obwody2006['latlon'].apply(split_lat)
obwody2006['lon']=obwody2006['latlon'].apply(split_lon)

In [65]:
obwody2010['lat']=obwody2010['latlon'].apply(split_lat)
obwody2010['lon']=obwody2010['latlon'].apply(split_lon)

In [66]:
obwody2014['lat']=obwody2014['latlon'].apply(split_lat)
obwody2014['lon']=obwody2014['latlon'].apply(split_lon)

In [68]:
obwody2006=obwody2006.drop('latlon', axis=1)
obwody2010=obwody2010.drop('latlon', axis=1)
obwody2014=obwody2014.drop('latlon', axis=1)

In [ ]:
obwody2006.head()

In [ ]:
obwody2006.to_csv("obwody2006_latlon.csv")

### Web scraping - wyniki wyborów

#### wyniki -wybory prezydenckie 2006

In [243]:
results_president2006=[]
page=requests.get("http://wybory2006.pkw.gov.pl/kbw/wynikiWbp91f5.html?instytucja=126101-WBP&strona=0")
for page_nr in range(23):
    tree=html.fromstring(page.content)
    #print(page.content)
    links=tree.xpath("//*[contains(text(),'Obwód')]")
    links_text=[link.get('href') for link in links]
    next_page_link=tree.xpath("//*[contains(text(),'Następna strona')]")
    for link in links_text:
        subpage=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+link)    
        subtree=html.fromstring(subpage.content)    
        nr_obwodu=subtree.xpath("//*[contains(text(),'Obwód nr')]")    
        nr_obwodu=nr_obwodu[0].text.split(" ")
        nr_obwodu=nr_obwodu[2]
        stat=subtree.xpath("//table[@id='stats']//td")
        candidates_data=subtree.xpath("//table[@id='kandydaci']//td")    
        max_voters=int(stat[7].text.replace("\xa0",""))
        votes=int(stat[13].text.replace("\xa0",""))
        majchrowski=int(candidates_data[19].text.replace(" ",""))
        pis= int(candidates_data[33].text.replace(" ",""))
        po=int(candidates_data[26].text.replace(" ",""))
        inni =votes-majchrowski-pis-po
        results_president2006.append([2006, nr_obwodu,majchrowski,po,pis,0,0,0,inni,max_voters,votes])
    if len(next_page_link)>0:
        page=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+next_page_link[0].get('href'))
    
    
        
    

In [244]:
results_president2006

[[2006, '1', 213, 117, 121, 0, 0, 0, 47, 1253, 498],
 [2006, '2', 144, 79, 130, 0, 0, 0, 39, 1074, 392],
 [2006, '3', 183, 116, 110, 0, 0, 0, 56, 1421, 465],
 [2006, '4', 179, 90, 115, 0, 0, 0, 41, 1180, 425],
 [2006, '5', 203, 107, 125, 0, 0, 0, 54, 1291, 489],
 [2006, '6', 144, 58, 89, 0, 0, 0, 41, 835, 332],
 [2006, '7', 249, 149, 200, 0, 0, 0, 69, 1558, 667],
 [2006, '8', 219, 148, 196, 0, 0, 0, 61, 1650, 624],
 [2006, '9', 133, 105, 91, 0, 0, 0, 35, 920, 364],
 [2006, '10', 116, 120, 167, 0, 0, 0, 48, 1056, 451],
 [2006, '11', 183, 120, 148, 0, 0, 0, 55, 1248, 506],
 [2006, '12', 169, 130, 95, 0, 0, 0, 78, 1085, 472],
 [2006, '13', 256, 151, 161, 0, 0, 0, 49, 1540, 617],
 [2006, '14', 166, 99, 83, 0, 0, 0, 37, 862, 385],
 [2006, '15', 187, 157, 203, 0, 0, 0, 54, 1184, 601],
 [2006, '16', 238, 164, 214, 0, 0, 0, 71, 1365, 687],
 [2006, '17', 170, 126, 116, 0, 0, 0, 53, 973, 465],
 [2006, '18', 256, 158, 134, 0, 0, 0, 64, 1239, 612],
 [2006, '19', 153, 124, 186, 0, 0, 0, 47, 1051, 5

#### wyniki - wybory do rady miasta 2006

In [245]:
page = requests.get("http://wybory2006.pkw.gov.pl/kbw/wybRadaMiasta9ff4.html?jdn=126101")
tree=html.fromstring(page.content)
okregi=tree.xpath("//*[contains(text(),'Okręg wyborczy nr')]")

In [246]:
def results(div):
    table=div.xpath("./table/tbody")
    result=0
    rows=table[0].xpath("./tr")
    for row in rows:
        try:
            result+=int(row.xpath("./td")[3].text.replace(" ",""))       
        except:
            result+=0
    return result


In [247]:
results_rada2006=[]
for okreg in okregi:
    subpage=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+okreg.get("href"))
    subtree=html.fromstring(subpage.content)
    link=subtree.xpath('//*[contains(text(),"Wyniki głosowania i wyniki wyborów w okręgu wyborczym")]')
    subpage2=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+link[0].get("href"))
    subtree2=html.fromstring(subpage2.content)
    obwody=subtree2.xpath('//*[contains(text(), "Obwód nr")]')
    next_page=[0]
    while len(next_page)>0:
        if next_page[0]!=0:
            subpage2=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+next_page[0].get("href"))
            subtree2=html.fromstring(subpage2.content)
            obwody=subtree2.xpath('//*[contains(text(), "Obwód nr")]')
        for obwod in obwody:
            subpage3=requests.get("http://wybory2006.pkw.gov.pl/kbw/"+obwod.get("href"))
            subtree3=html.fromstring(subpage3.content)
            obwod_nr=subtree3.xpath('//*[contains(text(), "Obwód nr")]')[0].text.split(" ")[2]
            max_voters=int(subtree3.xpath('//table[@id="stats"]/tbody//td')[9].text.replace("\xa0",""))
            votes=int(subtree3.xpath('//table[@id="stats"]/tbody//td')[15].text.replace("\xa0",""))
            result_divs=subtree3.xpath('//div[@class="block"]')
            for div in result_divs:
                h3=div.xpath("./h3")                
                if len(h3)>0:
                    #print(obwod_nr)
                    party=h3[0].text                        
                    if party=="Komitet Wyborczy Platforma Obywatelska RP":
                        po=results(div)                       
                    elif party=="Komitet Wyborczy Prawo i Sprawiedliwość":
                        pis=results(div)
                    elif party=="Koalicyjny Komitet Wyborczy SLD+SDPL+PD+UP Lewica i Demokraci":
                        left=results(div)
                    elif party=="Komitet Wyborczy Wyborców Jacka Majchrowskiego":
                        majchrowski=results(div)       
            inni=votes-po-pis-left-majchrowski
            results_rada2006.append([2006, obwod_nr,majchrowski,po,pis,0,left,0,inni,max_voters,votes])
        next_page=subtree2.xpath('//*[contains(text(), "Następna strona")]')

In [248]:
df_res_pres2006=pd.DataFrame.from_records(results_president2006,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lesniak","berkowicz","inni","max_voters","votes"])

In [249]:
df_res_pres2006.head()

,year,nr_obwodu,majchrowski,po,pis,gibala,lesniak,berkowicz,inni,max_voters,votes
0,2006,1,213,117,121,0,0,0,47,1253,498
1,2006,2,144,79,130,0,0,0,39,1074,392
2,2006,3,183,116,110,0,0,0,56,1421,465
3,2006,4,179,90,115,0,0,0,41,1180,425
4,2006,5,203,107,125,0,0,0,54,1291,489


In [250]:
df_res_pres2006[['majchrowski','po','pis','inni','max_voters','votes']].sum()

majchrowski    103157
po              55779
pis             63860
inni            20989
max_voters     586119
votes          243785
dtype: int64

In [251]:
df_res_ra2006=pd.DataFrame.from_records(results_rada2006,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lewica","berkowicz","inni","max_voters","votes"])

In [252]:
df_res_ra2006.head()

,year,nr_obwodu,majchrowski,po,pis,gibala,lewica,berkowicz,inni,max_voters,votes
0,2006,1,59,171,133,0,47,0,79,1253,489
1,2006,2,30,121,131,0,36,0,60,1074,378
2,2006,3,37,164,119,0,34,0,97,1421,451
3,2006,4,35,134,117,0,46,0,78,1180,410
4,2006,5,54,168,136,0,48,0,74,1291,480


In [253]:
df_res_ra2006[['majchrowski','po','pis','lewica','inni','max_voters','votes']].sum()

majchrowski     26630
po              90331
pis             68323
lewica          19217
inni            34276
max_voters     586119
votes          238777
dtype: int64

#### wyniki - wybory prezydenta 2010

In [254]:
page= requests.get("http://wybory2010.pkw.gov.pl/geo/pl/120000/126101.html#tabs-6")
tree=html.fromstring(page.content)


In [255]:
results_prezydent2010=[]
for obwod_nr in range(1,460):
    subpage=requests.get("http://wybory2010.pkw.gov.pl/obwwbp/pl/1261/126101-"+str(obwod_nr)+".html")
    subtree=html.fromstring(subpage.content)
    max_voters=int(subtree.xpath('//*[@id="text"]/div[2]/table[1]/tr[5]/td[2]')[0].text.replace("\xa0",""))
    votes=int(subtree.xpath('//*[@id="text"]/div[2]/table[1]/tr[8]/td[2]')[0].text.replace("\xa0",""))             
    majchrowski=int(subtree.xpath('//*[@id="text"]/div[2]/table[2]/tr[6]/td[6]')[0].text.replace(" ",""))
    po=int(subtree.xpath('//*[@id="text"]/div[2]/table[2]/tr[5]/td[6]')[0].text.replace(" ",""))
    pis=int(subtree.xpath('//*[@id="text"]/div[2]/table[2]/tr[3]/td[6]')[0].text.replace(" ",""))
    inni=votes-pis-po-majchrowski
    results_prezydent2010.append([2010,obwod_nr,majchrowski,po,pis,0,0,0,inni,max_voters, votes])
    

In [256]:
results_prezydent2010

[[2010, 1, 190, 154, 97, 0, 0, 0, 22, 1103, 463],
 [2010, 2, 138, 123, 97, 0, 0, 0, 19, 998, 377],
 [2010, 3, 170, 136, 85, 0, 0, 0, 16, 1239, 407],
 [2010, 4, 150, 115, 89, 0, 0, 0, 23, 1063, 377],
 [2010, 5, 172, 147, 80, 0, 0, 0, 18, 1158, 417],
 [2010, 6, 126, 84, 42, 0, 0, 0, 14, 704, 266],
 [2010, 7, 242, 167, 216, 0, 0, 0, 17, 1498, 642],
 [2010, 8, 203, 201, 163, 0, 0, 0, 37, 1464, 604],
 [2010, 9, 123, 105, 61, 0, 0, 0, 9, 786, 298],
 [2010, 10, 124, 111, 131, 0, 0, 0, 12, 844, 378],
 [2010, 11, 156, 149, 107, 0, 0, 0, 14, 1043, 426],
 [2010, 12, 136, 157, 115, 0, 0, 0, 25, 927, 433],
 [2010, 13, 267, 194, 147, 0, 0, 0, 20, 1385, 628],
 [2010, 14, 105, 127, 69, 0, 0, 0, 11, 721, 312],
 [2010, 15, 162, 203, 154, 0, 0, 0, 14, 1059, 533],
 [2010, 16, 191, 259, 154, 0, 0, 0, 35, 1264, 639],
 [2010, 17, 155, 148, 91, 0, 0, 0, 22, 877, 416],
 [2010, 18, 196, 193, 94, 0, 0, 0, 27, 1026, 510],
 [2010, 19, 170, 147, 147, 0, 0, 0, 14, 931, 478],
 [2010, 20, 128, 122, 50, 0, 0, 0, 14, 69

In [257]:
df_res_pres2010=pd.DataFrame.from_records(results_prezydent2010,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lesniak","berkowicz","inni","max_voters","votes"])

In [258]:
df_res_pres2010.sum()

year           922590
nr_obwodu      105570
majchrowski    102594
po              84748
pis             56302
gibala              0
lesniak             0
berkowicz           0
inni             7919
max_voters     586072
votes          251563
dtype: int64

#### wyniki - wybory rada 2010

In [260]:
results_rada2010=[]
for okreg_nr in range(1,8):
    page = requests.get("http://wybory2010.pkw.gov.pl/geo/pl/120000/126101-o-126101-RDA-"+str(okreg_nr)+".html?wyniki=1#tabs-2")
    tree=html.fromstring(page.content)
    obwody=tree.xpath('//*[@id="okr126101-RDA-'+str(okreg_nr)+'-obw-list"]/table/tr/td/u/a')
    for obwod in obwody:
        subpage=requests.get("http://wybory2010.pkw.gov.pl"+obwod.get('href'))
        subtree=html.fromstring(subpage.content)
        obwod_nr=subtree.xpath("//*[contains(text(),'Obwód nr')]")[0].text.split(" ")[2]
        max_voters=int(subtree.xpath('//*[@id="right"]/div[3]/table[1]/tr[6]/td[2]')[0].text.replace("\xa0",""))
        votes=int(subtree.xpath('//*[@id="right"]/div[3]/table[1]/tr[9]/td[2]')[0].text.replace("\xa0",""))
        majchrowski=int(subtree.xpath('//*[@id="right"]/div[3]/table[tr/td//text()[contains(., "Lista nr 17")]]/tr[@class="total"]/td[2]')[0].text.replace(" ",""))
        po=int(subtree.xpath('//*[@id="right"]/div[3]/table[3]/tr[@class="total"]/td[2]')[0].text.replace(" ",""))
        pis=int(subtree.xpath('//*[@id="right"]/div[3]/table[4]/tr[@class="total"]/td[2]')[0].text.replace(" ",""))
        left=int(subtree.xpath('//*[@id="right"]/div[3]/table[2]/tr[@class="total"]/td[2]')[0].text.replace(" ",""))
        inni=votes-po-pis-majchrowski-left
        results_rada2010.append([2010,obwod_nr,majchrowski,po,pis,0,left,0,inni,max_voters,votes])        
                
        

KeyboardInterrupt: 

In [ ]:
results_rada2010 = results_rada2010[::2]

In [ ]:
results_rada2010

In [ ]:
df_res_ra2010=pd.DataFrame.from_records(results_rada2010,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lewica","berkowicz","inni","max_voters","votes"])

In [ ]:
df_res_ra2010.sum()

#### wyniki - wybory prezydent 2014

In [261]:
page=requests.get("http://samorzad2014.pkw.gov.pl/360_Wybory_Burmistrza_-_I_tura/0/126101")
tree=html.fromstring(page.content)
obwody=tree.xpath('//table[@class="jstable"][1]/tbody/tr/td/a')[::5]

In [262]:
results_prezydent2014=[]
for obwod in obwody:   
    subpage=requests.get("http://samorzad2014.pkw.gov.pl/"+obwod.get('href')+"/wbp1_1")
    subtree=html.fromstring(subpage.content)    
    obwod_nr=subtree.xpath('//*[@class="kom_karta_tab"]/div[2]/div[2]')[0].text
    max_voters=int(subtree.xpath('//div[@class="kom_karta_tab2"]/div[1]/div[3]')[0].text)
    votes=int(subtree.xpath('//*[@id="kom_karta"]/div[3]/div[2]/div/div[25]/div[3]')[0].text)
    majchrowski=int(subtree.xpath('//tr[td="Majchrowski Jacek Maria"]/td[3]')[0].text.replace(" ",""))
    po=int(subtree.xpath('//tr[td="Patena Marta Elżbieta"]/td[3]')[0].text.replace(" ",""))
    pis=int(subtree.xpath('//tr[td="Lasota Marek Wojciech"]/td[3]')[0].text.replace(" ",""))
    gibala=int(subtree.xpath('//tr[td="Gibała Łukasz"]/td[3]')[0].text.replace(" ",""))  
    lesniak=int(subtree.xpath('//tr[td="Leśniak Tomasz Marek"]/td[3]')[0].text.replace(" ",""))
    berkowicz=int(subtree.xpath('//tr[td="Berkowicz Konrad Szczepan"]/td[3]')[0].text.replace(" ",""))
    inni=votes-po-pis-majchrowski-gibala-lesniak-berkowicz
    results_prezydent2014.append([2014,obwod_nr,majchrowski,po,pis,gibala,lesniak,berkowicz,inni,max_voters,votes])

In [263]:
results_prezydent2014

[[2014, '1', 155, 27, 103, 36, 23, 17, 25, 1008, 386],
 [2014, '10', 101, 26, 170, 27, 12, 4, 31, 733, 371],
 [2014, '100', 284, 66, 193, 80, 39, 26, 29, 1600, 717],
 [2014, '101', 201, 68, 88, 70, 45, 31, 30, 1107, 533],
 [2014, '102', 209, 49, 112, 58, 39, 32, 24, 1197, 523],
 [2014, '103', 242, 57, 114, 76, 23, 23, 25, 1288, 560],
 [2014, '104', 181, 44, 47, 66, 37, 18, 19, 954, 412],
 [2014, '105', 152, 51, 91, 45, 22, 15, 12, 777, 388],
 [2014, '106', 385, 80, 207, 97, 57, 26, 28, 1924, 880],
 [2014, '107', 170, 39, 120, 75, 39, 14, 19, 1085, 476],
 [2014, '108', 206, 56, 96, 82, 25, 24, 16, 1106, 505],
 [2014, '109', 268, 66, 203, 94, 59, 21, 35, 1554, 746],
 [2014, '11', 129, 28, 106, 32, 19, 11, 15, 848, 340],
 [2014, '110', 165, 76, 157, 55, 26, 29, 18, 1113, 526],
 [2014, '111', 374, 94, 173, 111, 42, 26, 28, 1938, 848],
 [2014, '112', 403, 73, 165, 100, 55, 22, 31, 1985, 849],
 [2014, '113', 373, 81, 209, 81, 44, 30, 20, 1857, 838],
 [2014, '114', 329, 74, 184, 91, 33, 35, 1

In [264]:
df_res_pres2014=pd.DataFrame.from_records(results_prezydent2014,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lesniak","berkowicz","inni","max_voters","votes"])

In [265]:
df_res_pres2014[["majchrowski","po","pis","gibala","lesniak","berkowicz","inni"]].sum()

majchrowski    95106
po             22267
pis            65417
gibala         27090
lesniak        11749
berkowicz      10489
inni           10653
dtype: int64

In [268]:
df_res_pres2014['nr_obwodu']=df_res_pres2014['nr_obwodu'].astype("int")
df_res_pres2014=df_res_pres2014.sort_values(["nr_obwodu"])
df_res_pres2014.head()

,year,nr_obwodu,majchrowski,po,pis,gibala,lesniak,berkowicz,inni,max_voters,votes
0,2014,1,155,27,103,36,23,17,25,1008,386
111,2014,2,131,31,109,22,21,11,22,942,347
222,2014,3,133,34,104,43,24,12,20,1125,370
333,2014,4,126,28,78,41,18,18,17,909,326
432,2014,5,128,37,83,32,17,22,18,1039,337


#### wyniki - wybory rada 2014

In [269]:
page=requests.get("http://samorzad2014.pkw.gov.pl/359_Rady_gmin/0/126101")
tree=html.fromstring(page.content)
obwody=tree.xpath('//table[@class="jstable"][1]/tbody/tr/td/a')[::5]

In [270]:
len(obwody)

487

In [271]:
results_rada2014=[]
for obwod in obwody:   
    subpage=requests.get("http://samorzad2014.pkw.gov.pl/"+obwod.get('href'))    
    subtree=html.fromstring(subpage.content)    
    link=subtree.xpath("//*[contains(text(),'Rada gminy - okręg:')]")
    subpage=requests.get("http://samorzad2014.pkw.gov.pl/"+link[0].get('href'))    
    subtree=html.fromstring(subpage.content)       
    obwod_nr=subtree.xpath('//*[@class="kom_karta_tab"]/div[2]/div[2]')[0].text
    max_voters=int(subtree.xpath('//div[@class="kom_karta_tab2"]/div[1]/div[3]')[0].text)
    votes=int(subtree.xpath('//*[@id="kom_karta"]/div[3]/div[2]/div/div[22]/div[3]')[0].text)
    majchrowski=int(subtree.xpath('//tr[td="KWW Jacka Majchrowskiego"][1]/td[3]')[0].text.replace(" ",""))
    po=int(subtree.xpath('//tr[td="KW Platforma Obywatelska RP"]/td[3]')[0].text.replace(" ",""))
    pis=int(subtree.xpath('//tr[td="KW Prawo i Sprawiedliwość"]/td[3]')[0].text.replace(" ",""))
    gibala=int(subtree.xpath('//tr[td//text()[contains(., "KWW Łukasza Gibały")]]/td[3]')[0].text.replace(" ",""))  
    left=int(subtree.xpath('//tr[td="KWW Kraków Przeciw Igrzyskom"]/td[3]')[0].text.replace(" ",""))
    berkowicz=int(subtree.xpath('//tr[td="KWW Nowa Prawica - Kraków"]/td[3]')[0].text.replace(" ",""))
    inni=votes-po-pis-majchrowski-gibala-left-berkowicz
    results_rada2014.append([2014,obwod_nr,majchrowski,po,pis,gibala,left,berkowicz,inni,max_voters,votes])

In [272]:
len(results_rada2014)

487

In [273]:
df_res_ra2014=pd.DataFrame.from_records(results_rada2014,columns=["year","nr_obwodu","majchrowski","po","pis","gibala","lewica","berkowicz","inni","max_voters","votes"])

In [274]:
df_res_ra2014[["majchrowski","po","pis","gibala","lewica","berkowicz","inni"]].sum().sum()

228864

In [275]:
df_res_ra2014[["max_voters","votes"]].sum()

max_voters    586233
votes         228864
dtype: int64

In [276]:
df_res_ra2014['nr_obwodu']=df_res_ra2014['nr_obwodu'].astype("int")
df_res_ra2014=df_res_ra2014.sort_values(["nr_obwodu"])
df_res_ra2014.head()

,year,nr_obwodu,majchrowski,po,pis,gibala,lewica,berkowicz,inni,max_voters,votes
0,2014,1,52,106,110,16,27,23,33,1008,367
111,2014,2,40,93,118,6,23,17,27,942,324
222,2014,3,42,107,112,17,31,14,29,1125,352
333,2014,4,47,70,100,12,22,19,32,909,302
432,2014,5,42,95,99,8,26,22,26,1039,318


### wyniki wyborów- data cleaning

1. Usunięcie obwodów zamkniętych

In [277]:
df_res_pres2006['nr_obwodu']=df_res_pres2006['nr_obwodu'].astype('int')
df_res_pres2006=df_res_pres2006[df_res_pres2006['nr_obwodu'].isin(obwody2006['nr'])]
df_res_ra2006['nr_obwodu']=df_res_ra2006['nr_obwodu'].astype('int')
df_res_ra2006=df_res_ra2006[df_res_ra2006['nr_obwodu'].isin(obwody2006['nr'])]
df_res_pres2010=df_res_pres2010[df_res_pres2010['nr_obwodu'].isin(obwody2010['nr'])]


In [278]:
df_res_ra2010['nr_obwodu']=df_res_ra2010['nr_obwodu'].astype('int')

In [279]:
df_res_ra2010=df_res_ra2010[df_res_ra2010['nr_obwodu'].isin(obwody2010['nr'])]


In [280]:
len(df_res_ra2010)

424

In [281]:
df_res_pres2014['nr_obwodu']=df_res_pres2014['nr_obwodu'].astype('int')
df_res_pres2014=df_res_pres2014[df_res_pres2014['nr_obwodu'].isin(obwody2014['nr'])]

In [282]:
df_res_ra2014['nr_obwodu']=df_res_ra2014['nr_obwodu'].astype('int')
df_res_ra2014=df_res_ra2014[df_res_ra2014['nr_obwodu'].isin(obwody2014['nr'])]

In [283]:
len(df_res_pres2014)

452

In [284]:
len(df_res_ra2014)

452

2.Dodanie mapowania obwodów na obwody 2006

In [285]:
df_res_pres2010['obwod06']=obwody2010['obwod06']

In [286]:
df_res_ra2010['obwod06']=obwody2010['obwod06']
print(len(obwody2010[obwody2010['obwod06'].isnull()]))
len(df_res_ra2010[df_res_ra2010["obwod06"].isnull()])
df_res_ra2010[df_res_ra2010["obwod06"].isnull()==True]
obwody2010.tail(30)
df_res_pres2010.tail(40)

0


,year,nr_obwodu,majchrowski,po,pis,gibala,lesniak,berkowicz,inni,max_voters,votes,obwod06
384,2010,385,169,145,109,0,0,0,6,1064,429,374
385,2010,386,185,201,198,0,0,0,13,1402,597,375
386,2010,387,108,113,69,0,0,0,2,759,292,376
387,2010,388,299,239,174,0,0,0,9,1873,721,377
388,2010,389,159,118,92,0,0,0,10,1117,379,378
389,2010,390,103,243,88,0,0,0,3,1365,437,379
390,2010,391,393,244,177,0,0,0,38,2092,852,380
391,2010,392,248,198,181,0,0,0,15,1676,642,381
392,2010,393,218,148,117,0,0,0,17,1311,500,382
393,2010,394,252,203,102,0,0,0,13,1349,570,383


In [287]:
df_res_pres2014['obwod06']=obwody2014['obwod06']

In [288]:
df_res_ra2014['obwod06']=obwody2014['obwod06']

4.Dodanie wyniku względnego

In [3]:
def pres(df):
    df['majchrowski_proc']=df['majchrowski']/df['votes']
    df['po_proc']=df['po']/df['votes']
    df['pis_proc']=df['pis']/df['votes']
    df['gibala_proc']=df['gibala']/df['votes']
    df['lesniak_proc']=df['lesniak']/df['votes']
    df['berkowicz_proc']=df['berkowicz']/df['votes']
    df['inni_proc']=df['inni']/df['votes']
    return df

def rada(df):
    df['majchrowski_proc']=df['majchrowski']/df['votes']
    df['po_proc']=df['po']/df['votes']
    df['pis_proc']=df['pis']/df['votes']
    df['gibala_proc']=df['gibala']/df['votes']
    df['lewica_proc']=df['lewica']/df['votes']
    df['berkowicz_proc']=df['berkowicz']/df['votes']
    df['inni_proc']=df['inni']/df['votes']
    return df

df_res_pres2006=pres(df_res_pres2006)
df_res_pres2010=pres(df_res_pres2010)
df_res_pres2014=pres(df_res_pres2014)
df_res_ra2006=rada(df_res_ra2006)
df_res_ra2010=rada(df_res_ra2010)
df_res_ra2014=rada(df_res_ra2014)

NameError: name 'df_res_pres2006' is not defined

3.Zapis do pliku

In [69]:
#df_res_pres2006.to_csv("res_pres2006.csv", encoding="UTF-8")
#df_res_ra2006.to_csv("res_ra2006.csv", encoding="UTF-8")
#df_res_pres2010.to_csv("res_pres2010.csv", encoding="UTF-8")
#df_res_ra2010.to_csv("res_ra2010.csv", encoding="UTF-8")
#df_res_pres2014.to_csv("res_pres2014.csv", encoding="UTF-8")
#df_res_ra2014.to_csv("res_ra2014.csv", encoding="UTF-8")
#obwody2006.to_csv("obwody2006_clean.csv", encoding="UTF-8")
obwody2010.to_csv("obwody2010_clean.csv", encoding="UTF-8")
obwody2014.to_csv("obwody2014_clean.csv", encoding="UTF-8")